<a href="https://colab.research.google.com/github/pawansharmaaaa/Wav2Lip_Mod/blob/master/GFPGAN_Wav2Lip_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prerequisites

Initialization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir /content/Data

In [3]:
!cp -ri "/content/drive/MyDrive/Wav2Lip_Mod/TAudio.wav" "/content/drive/MyDrive/Wav2Lip_Mod/input_vid.mp4" /content/Data

Wav2Lip

In [4]:
!git clone https://github.com/pawansharmaaaa/Wav2Lip_Mod.git

Cloning into 'Wav2Lip_Mod'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 399 (delta 19), reused 1 (delta 0), pack-reused 360
Receiving objects: 100% (399/399), 551.75 KiB | 16.23 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [5]:
!cp -ri "/content/drive/MyDrive/Wav2Lip_Mod/wav2lip.pth" "/content/drive/MyDrive/Wav2Lip_Mod/wav2lip_gan.pth" /content/Wav2Lip_Mod/checkpoints

In [6]:
!pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip_Mod && pip install -r requirements.txt

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 20.3 MB/s 

In [7]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip_Mod/face_detection/detection/sfd/s3fd.pth"

--2023-07-21 22:07:13--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘Wav2Lip_Mod/face_detection/detection/sfd/s3fd.pth’

Wav2Lip_Mod/face_de 100%[===================>]  85.68M  13.4MB/s    in 7.6s    

2023-07-21 22:07:22 (11.3 MB/s) - ‘Wav2Lip_Mod/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]



GFPGAN

In [ ]:
!cd Wav2Lip_Mod && git clone https://github.com/TencentARC/GFPGAN.git

In [ ]:
!pip install basicsr facexlib

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && pip install -r requirements.txt

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && python setup.py develop

In [ ]:
!pip install realesrgan

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && python inference_gfpgan.py -i /content/GFPGAN/inputs/test2.jpg -o results/new -v 1.3 -s 2

In [ ]:
!pip install gfpgan

In [2]:
!cp -ri "/content/Data/w2l_input_video.mp4" /content/drive/MyDrive/Finalized_notebook

#*Remove Frames Which do not contain faces from the Video*

In [ ]:
import cv2
import face_recognition
import gc
import tqdm

# Function to detect face in a frame
def detect_face(frames):
    batch_face_locations = face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0)
    return batch_face_locations

# Open the video file for reading
video_path = '/content/Data/linus.mp4'
video_capture = cv2.VideoCapture(video_path)

# Get video properties
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writers for face.mp4
face_writer = cv2.VideoWriter('/content/Data/w2l_input_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

batch_size = 16
frames = []
while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # Detect faces in the frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

    if len(frames) == batch_size:
      batch_faces = detect_face(frames)

      for frame_no, face_locations in enumerate(tqdm.tqdm(batch_faces, total=batch_size)):
        if len(face_locations) == 1:
            # Write the frame to input_video.mp4 if a face is detected
            pos_frame = cv2.cvtColor(frames[frame_no], cv2.COLOR_RGB2BGR)
            face_writer.write(pos_frame)
      frames = []

# Release video capture, writers and clear garbage
video_capture.release()
face_writer.release()
cv2.destroyAllWindows()
gc.collect()

#Running Wav2Lip

In [ ]:
!cd Wav2Lip_Mod && python inference_wgan.py --checkpoint_path checkpoints/wav2lip.pth --face "/content/Data/w2l_input_video.mp4" --audio "/content/Data/TAudio.wav" --resize_factor 2 --upscale --nosmooth

#FRAMER.py

In [9]:
import sys
sys.path.append('/content/GFPGAN')
!cd GFPGAN

In [ ]:
!cd GFPGAN
import cv2
import os
from gfpgan import GFPGANer
import torch


def divide_video_into_frames(video_path, output_dir):
    #LOAD REALESRGAN
    upsample_bg = 'realesrgan'
    if upsample_bg == 'realesrgan':
        if not torch.cuda.is_available():  # CPU
            import warnings
            warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
                          'If you really want to use it, please modify the corresponding codes.')
            bg_upsampler = None
        else:
            from basicsr.archs.rrdbnet_arch import RRDBNet
            from realesrgan import RealESRGANer
            model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
            bg_upsampler = RealESRGANer(
                scale=2,
                model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
                model=model,
                tile=0,
                tile_pad=10,
                pre_pad=0,
                half=True)  # need to set False in CPU mode
    else:
        bg_upsampler = None

    #LOAD GFPGAN
    arch = 'clean'
    channel_multiplier = 2
    model_name = 'GFPGANv1.3'
    url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'

    restorer = GFPGANer(
        model_path=url,
        upscale=2,
        arch=arch,
        channel_multiplier=channel_multiplier,
        bg_upsampler=bg_upsampler)

    # Get the video capture object
    video_capture = cv2.VideoCapture(video_path)

    # Get the number of frames in the video
    num_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Frames to improve: {num_frames}")

    # Iterate over the frames in the video
    for i in range(num_frames):
        # Read the frame from the video
        success, frame = video_capture.read()

        # If the frame was read successfully, save it
        if success:
            frame_name = os.path.join(output_dir, str(i) + ".jpg")
            cropped_faces, restored_faces, restored_img = restorer.enhance(
              frame,
              has_aligned=False,
              only_center_face=False,
              paste_back=True,
              weight=0.5)

            cv2.imwrite(frame_name, restored_img)

    # Close the video capture object
    video_capture.release()

if __name__ == "__main__":
    # Set the video path and output directory
    video_path = "/content/result_voice_theek.mp4"
    output_dir = "/content/Frames"

    # Divide the video into frames
    divide_video_into_frames(video_path, output_dir)